In [1]:
%%writefile mapper.py

import sys
import re

from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8')

path_to_file='stop_words_en.txt'
# path_to_file='/datasets/stop_words_en.txt'

with open(path_to_file) as stop_words_file:
    content = stop_words_file.readlines()
    stop_words = set(l.strip().lower() for l in content)

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = [w.lower() for w in re.split("\W*\s+\W*", text, flags=re.UNICODE) if w.lower() not in stop_words]
    all_wards = float(len(words))
    counters = Counter(words)
        
    tf = {w: counters[w] / all_wards for w in counters}
    
    for word in tf:
        print "%s\t%0.5f\t%s" % (word.lower(), tf[word], article_id)

Writing mapper.py


In [2]:
%%writefile reducer.py

import sys

from math import log

current_key = None
all_documents = 0
current_tf = 0.

for line in sys.stdin:
    try:
        key, tf, article_id = line.strip().split('\t', 2)
        article_id = int(article_id)
    except ValueError as e:
        continue
        
    if key == 'labor':
        all_documents += 1
        
    if article_id == 12:
        current_tf = float(tf)
        
    if current_key != key:
        if current_key == 'labor':
            idf = 1. / log(1. + all_documents)
            print "%0.10f" % (current_tf * idf)
            
        current_key = key
        all_documents = 0
        current_tf = 0

if current_key and current_key == 'labor':
    idf = 1. / log(1 + all_documents)
    print "%0.10f" % (current_tf * idf)


Writing reducer.py


In [3]:
%%bash

OUT_DIR="stop_words_result"
NUM_REDUCERS=5

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming stop words" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00004 | head


0.0003509630	


rm: `stop_words_result': No such file or directory
18/02/03 09:30:26 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/02/03 09:30:26 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/02/03 09:30:28 INFO mapred.FileInputFormat: Total input files to process : 1
18/02/03 09:30:29 INFO mapreduce.JobSubmitter: number of splits:2
18/02/03 09:30:29 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1517597425666_0001
18/02/03 09:30:29 INFO impl.YarnClientImpl: Submitted application application_1517597425666_0001
18/02/03 09:30:30 INFO mapreduce.Job: The url to track the job: http://2b93fde4fee3:8088/proxy/application_1517597425666_0001/
18/02/03 09:30:30 INFO mapreduce.Job: Running job: job_1517597425666_0001
18/02/03 09:30:36 INFO mapreduce.Job: Job job_1517597425666_0001 running in uber mode : false
18/02/03 09:30:36 INFO mapreduce.Job:  map 0% reduce 0%
18/02/03 09:30:52 INFO mapreduce.Job:  map 100% reduce 0%
18/02/03 09:30:58 INFO mapre